#Capstone Project- The Battle of Neighborhoods

    ###Introduction

Our goal is to find the best location in Thessaloniki, Greece where he can open a vegan restaurant.

Veganism is becoming more and more popular nowadays. The reason for these are that more people are becoming health conscious and concerned about ethical eating. 
There are a lot of non-vegans who love vegan food. Those who genuinely love a healthy lifestyle, for instance. So, a good location to open the restaurant is near yoga and pilates studios.
Also we will make sure that we choose a highly visible location and a location not near other vegan restaurants.

###Data

####1.Geographic locations of vegan restaurants of Thessaloniki